<a href="https://colab.research.google.com/github/NhipCau/FastAutoAudioStrip-Name/blob/main/fastautostrip_name.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#GoogleDriveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#ライブラリをインストール
!pip install pydub
!pip install openpyxl
!pip install webrtcvad

In [ ]:
#メインコード
import webrtcvad
import openpyxl
import os
from pydub import AudioSegment

# ===== 設定 =====
input_audio_path = "/content/drive/MyDrive/Colab Notebooks/sample_voice.wav"       # 入力WAVファイル（完全パス）
excel_path = "/content/drive/MyDrive/Colab Notebooks/sample_list.xlsx"             # Excelファイル（完全パス）
output_dir = "/content/drive/MyDrive/Colab Notebooks/output_wavs"                  # 出力ディレクトリ（完全パス）

target_sample_rate = 48000                  # サンプルレート（Hz）
vad_aggressiveness = 3                      # VADの攻撃性（0〜3）
#vad_aggressivenessとは？：0 → 息継ぎも含めて「音声」と判定／　3 → 息継ぎや間は「無音」と判定し、2つのチャンクに分割される可能性あり／  1 または 2 がバランスが良く、「誤検出を減らしたい」場合は 2、「できるだけ多くの音声を拾いたい」場合は 1

silence_thresh = -55                        # 無音とみなすdB閾値(完全無音は-60db,背景ノイズがある場合は要調整)
min_silence_len_ms = 1700                   # 最小サイレンス長（ミリ秒）事前に設定されている無音秒数-前後バッファ秒
pre_buffer_sec = 1.5                        # 前バッファ（秒）
post_buffer_sec = 1.5                       # 後バッファ（秒）

filename_column_index = 1                   # ファイル名を取得する列（A列=0, B列=1, ...）
filename_start_row = 4                      # ファイル名を取得する開始行（1行目=0, 2行目=1, ...）

# ===== 出力フォルダ作成 =====
os.makedirs(output_dir, exist_ok=True)

# ===== Excelからファイル名を取得 =====
wb = openpyxl.load_workbook(excel_path)
sheet = wb.active
filenames = [
    row[filename_column_index]
    for row in sheet.iter_rows(min_row=filename_start_row, values_only=True)
    if row[filename_column_index]
]

# ===== 音声読み込みとサンプルレート変換 =====
audio = AudioSegment.from_wav(input_audio_path)
audio = audio.set_channels(1).set_frame_rate(target_sample_rate).set_sample_width(2)

# ===== webrtcvad用に生データを準備 =====
raw_audio = audio.raw_data
frame_duration = 30  # ms
frame_size = int(target_sample_rate * frame_duration / 1000) * 2  # 2 bytes/sample

frames = [
    (i * frame_duration, raw_audio[i * frame_size:(i + 1) * frame_size])
    for i in range(len(raw_audio) // frame_size)
]

# ===== VADで音声区間を検出 =====
vad = webrtcvad.Vad(vad_aggressiveness)
speech_segments = []
is_speech = False
segment_start = 0

for i, (timestamp, frame) in enumerate(frames):
    if len(frame) < frame_size:
        break
    if vad.is_speech(frame, target_sample_rate):
        if not is_speech:
            segment_start = timestamp
            is_speech = True
    else:
        if is_speech:
            segment_end = timestamp
            speech_segments.append((segment_start, segment_end))
            is_speech = False

if is_speech:
    speech_segments.append((segment_start, timestamp + frame_duration))

# ===== dB閾値でフィルタリング =====
filtered_segments = []
for start_ms, end_ms in speech_segments:
    segment = audio[start_ms:end_ms]
    if segment.dBFS >= silence_thresh:
        filtered_segments.append((start_ms, end_ms))

# ===== 最小サイレンス長でマージ =====
merged_segments = []
if filtered_segments:
    current_start, current_end = filtered_segments[0]
    for start_ms, end_ms in filtered_segments[1:]:
        if start_ms - current_end < min_silence_len_ms:
            current_end = end_ms
        else:
            merged_segments.append((current_start, current_end))
            current_start, current_end = start_ms, end_ms
    merged_segments.append((current_start, current_end))

# ===== バッファを加えて保存 =====
pre_buffer_ms = int(pre_buffer_sec * 1000)
post_buffer_ms = int(post_buffer_sec * 1000)

for i, (start_ms, end_ms) in enumerate(merged_segments):
    if i < len(filenames):
        start = max(0, start_ms - pre_buffer_ms)
        end = min(len(audio), end_ms + post_buffer_ms)
        chunk = audio[start:end]
        output_path = os.path.join(output_dir, f"{filenames[i]}.wav")
        chunk.export(output_path, format="wav")
        print(f"保存しました: {output_path}")
    else:
        print(f"ファイル名が不足しています（チャンク{i+1}）")

print("処理が完了しました。")

